# OpenAI Agents SDK 사용 팁

먼저 uv로 가상환경을 만들었다면 `uv add --dev ipykernel` 명령어로 ipykernel을 설치해야 합니다.

이 노트북에서는 OpenAI Agents SDK를 사용하는 데 도움이 되는 다양한 팁과 모범 사례를 다룹니다.

## 목차
1. OpenAI 이외의 다른 모델 사용하기

2. Tracing 기능으로 내 에이전트 모니터링하기
   - 기본 예제
   - 멀티 에이전트 예제



### 1. OpenAI 이외의 다른 모델 사용하기
- openai 패키지와 직접 연동되는 모델들은 OpenAI AsyncOpenAI 사용하면 간편하게 모델 변경 가능


In [5]:
from agents import Agent, Runner
import os

from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel
from dotenv import load_dotenv

load_dotenv()

#Reference: https://ai.google.dev/gemini-api/docs/openai
client = AsyncOpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Create agent with selected configuration
agent = Agent(
    name="여행 에이전트",
    instructions="당신은 훌륭한 여행 에이전트입니다.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
)

prompt = "강원도 여행지 추천해줘"
result = await Runner.run(agent, prompt)
print(result.final_output)



강원도는 아름다운 자연과 다양한 즐길 거리로 가득한 곳입니다. 어떤 종류의 여행을 선호하시는지에 따라 추천 코스가 달라질 수 있으니, 몇 가지 질문을 드려도 될까요?

1.  **여행 스타일**: 혼자 떠나는 여행인가요? 아니면 연인, 가족, 친구와 함께하는 여행인가요?
2.  **관심사**: 자연 풍경 감상, 액티비티, 맛집 탐방, 역사 유적지 방문 등 어떤 쪽에 더 관심이 있으신가요?
3.  **여행 시기**: 여행 예정인 시기는 언제인가요? 계절에 따라 추천 장소가 달라질 수 있습니다.
4.  **예산**: 숙박, 식사, 액티비티 등에 사용할 예산은 어느 정도인가요?
5.  **이동 수단**: 자가용을 이용하시나요? 아니면 대중교통을 이용하시나요?

위 질문에 대한 답변을 알려주시면, 더욱 맞춤화된 강원도 여행 코스를 추천해 드릴 수 있습니다. 😊

만약 답변이 어려우시다면, 일반적인 추천 코스를 몇 가지 소개해 드릴게요.

**추천 코스 (일반)**

*   **속초 & 고성**: 아름다운 해변과 설악산의 절경을 함께 즐길 수 있는 곳입니다.
    *   **주요 관광지**: 속초 해변, 외옹치 바다향기로, 설악산 국립공원, 대포항, 영금정, 고성 통일전망대, 화진포 해변
    *   **추천 활동**: 해변 산책, 설악산 케이블카 탑승, 싱싱한 해산물 맛보기
*   **강릉**: 역사와 문화, 아름다운 바다를 동시에 경험할 수 있는 곳입니다.
    *   **주요 관광지**: 경포 해변, 오죽헌, 선교장, 안목 해변 커피거리, 정동진
    *   **추천 활동**: 해변에서 일출 감상, 역사 유적지 탐방, 커피거리에서 커피 한잔
*   **평창 & 정선**: 산과 계곡의 아름다움을 만끽하고 액티비티를 즐길 수 있는 곳입니다.
    *   **주요 관광지**: 오대산 국립공원, 월정사, 대관령 양떼목장, 정선 레일바이크, 화암동굴
    *   **추천 활동**: 트레킹, 양떼목장 체험, 레일바이크 타기, 동굴 탐험
*   **춘천**: 아름다

###  2. Tracing 사용하기

- Tracing은 에이전트의 동작을 추적하고 모니터링하는 기능
- 에이전트가 어떤 작업을 수행하고 있는지, 어떤 결정을 내렸는지 확인할 수 있습니다.


In [ ]:
from agents import Agent, Runner

# 기본 에이전트 생성
agent = Agent(
    name="기본 에이전트",
    instructions="당신은 도움이 되는 한국어 어시스턴트입니다. 모든 응답을 대문자로 작성해주세요.",
    model="gpt-4"
)

# 에이전트 실행 - tracing이 자동으로 활성화됨
result = await Runner.run(agent, "안녕하세요! 오늘 기분이 어떠신가요?")
print(result.final_output)


In [ ]:

from agents import Agent, Runner, trace

# 두 개의 에이전트 생성
joke_agent = Agent(
    name="유머 에이전트",
    instructions="당신은 유머러스한 한국어 에이전트입니다. 주어진 주제에 대해 재미있는 농담을 만들어주세요.",
)

translator_agent = Agent(
    name="번역 에이전트",
    instructions="당신은 전문 번역가입니다. 주어진 텍스트를 영어로 번역해주세요.",
)


# case 1 trace 없는 경우

# 1단계: 농담 생성
joke_result = await Runner.run(joke_agent, "컴퓨터")
print("한국어 농담:", joke_result.final_output)

# 2단계: 농담 번역
translation_result = await Runner.run(
    translator_agent, 
    f"다음 농담을 영어로 번역해주세요: {joke_result.final_output}"
)
print("\n영어 번역:", translation_result.final_output)

# trace 컨텍스트 매니저를 사용한 워크플로우 추적
# with trace("유머 번역 워크플로우"):
#     # 1단계: 농담 생성
#     joke_result = await Runner.run(joke_agent, "컴퓨터")
#     print("한국어 농담:", joke_result.final_output)
    
#     # 2단계: 농담 번역
#     translation_result = await Runner.run(
#         translator_agent, 
#         f"다음 농담을 영어로 번역해주세요: {joke_result.final_output}"
#     )
#     print("\n영어 번역:", translation_result.final_output)
